<a href="https://colab.research.google.com/github/bensoudaneOsama/projet_Deep_learning/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Oct 22 07:40:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 install transformers
!pip3 install nerda

     |████████████████████████████████| 2.9 MB 10.9 MB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
     |████████████████████████████████| 895 kB 44.1 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=21e1ec7c0d17e2718b7345c3f8db786beb5572de8e966034880947e651b6f5fa
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar


In [ ]:
"""
Extracting the preprocessed data from the pkl files 
"""
import pickle

beth_tokens, beth_concepts = pickle.load(open('beth.pkl', 'rb'))
partners_tokens, partners_concepts = pickle.load(open('partners.pkl', 'rb'))
test_tokens, test_concepts = pickle.load(open('text.pkl', 'rb'))


In [ ]:
from sklearn.model_selection import train_test_split

train_dataset = {}
val_dataset = {}
test_dataset = {}

x = beth_tokens + partners_tokens
y = beth_concepts + partners_concepts

train_dataset['sentences'], val_dataset['sentences'], train_dataset['tags'], val_dataset['tags'] = train_test_split(x,y,test_size=0.1)

test_dataset['sentences'] = test_tokens
test_dataset['tags'] = test_concepts

for ele in train_dataset:
  print(f'{ele}:- Training {train_dataset[ele]}')
  print(f'{ele}:- Validation {val_dataset[ele]}')


sentences:- Training [['POTENTIALLY', 'SERIOUS', 'INTERACTION', ':'], ['D', ':', '2015-09-21', '08:34'], ['16.', 'medication'], ['ED', 'DISCHARGE', 'NOTIFICATION', '/', 'SUMMARY'], [''], ['On', 'the', 'day', 'of', 'discharge', ',', 'he', 'is', 'stating', 'approximately', '91', 'to', '92%', 'on', 'three', 'to', 'four', 'liters', '.'], ['She', 'was', 'empirically', 'started', 'on', 'vanco/zosyn', 'for', 'empiric', 'hosp', 'acquired', 'pna', 'coverage'], ['Neurological', 'examination', 'revealed', 'that', 'the', 'patient', 'was', 'quite', 'sedated', 'at', 'this', 'time', 'having', 'received', 'some', 'medications', '.'], ['2013-12-24', '01:30', 'AM'], ['PAST', 'MEDICAL', 'HISTORY', ':'], ['He', 'was', 'most', 'recently', 'discharged', 'from', 'Oaksgekesser/', 'Memorial', 'Hospital', 'on', '03/06/99', 'and', 'was', 'then', 'transferred', 'to', 'Linghs', 'County', 'Medical', 'Center', 'from', 'which', 'he', 'was', 'discharged', 'on', '03/20/99', '.'], ['T', ':', '2010-05-27', '14:10'], ['T'

In [ ]:
tag_scheme = ['problem', 'treatment', 'test', 'B-problem', 'B-treatment', 'B-test']
transformer = "emilyalsentzer/Bio_ClinicalBERT"

# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
'epochs' : 4,
'warmup_steps' : 500,
'train_batch_size': 10,                                         
'learning_rate': 0.0001
}

from NERDA.models import NERDA

model = NERDA(
  dataset_training = train_dataset,
  dataset_validation = val_dataset,
  tag_scheme = tag_scheme, 
  max_len = 256,
  tag_outside = '',
  transformer = transformer,
  dropout = dropout,
  hyperparameters = training_hyperparameters
)

model.train()
test = test_dataset
model.evaluate_performance(test)

Device automatically set to: cuda


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



 Epoch 1 / 4


 29%|██▉       | 425/1478 [06:17<15:37,  1.12it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #4277 length 272 exceeds max_len 256 and has been truncated
  warnings.warn(msg)
 85%|████████▌ | 1258/1478 [18:39<03:16,  1.12it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #12608 length 282 exceeds max_len 256 and has been truncated
  warnings.warn(msg)
100%|██████████| 206/206 [00:49<00:00,  4.15it/s]


Train Loss = 0.2807427490141409 Valid Loss = 0.16827227336182757

 Epoch 2 / 4


...

Train Loss = 0.049185686304627954 Valid Loss = 0.16870872057026062

 Epoch 4 / 4


 29%|██▉       | 425/1478 [06:20<15:42,  1.12it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #4277 length 272 exceeds max_len 256 and has been truncated
  warnings.warn(msg)
 85%|████████▌ | 1258/1478 [18:45<03:17,  1.12it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #12608 length 282 exceeds max_len 256 and has been truncated
  warnings.warn(msg)
100%|██████████| 206/206 [00:49<00:00,  4.13it/s]

Train Loss = 0.02316084486938914 Valid Loss = 0.17055891821627422



/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #4831 length 287 exceeds max_len 256 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14971 length 506 exceeds max_len 256 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #25037 length 275 exceeds max_len 256 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 591 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')


,Level,F1-Score,Precision,Recall
0,problem,0.899204,0.891408,0.907137
1,treatment,0.860257,0.861451,0.859066
2,test,0.873740,0.876428,0.871068
3,B-problem,0.913256,0.914602,0.911914
4,B-treatment,0.896908,0.900054,0.893783
5,B-test,0.897152,0.884534,0.910136
0,AVG_MICRO,0.893413,NaN,NaN
0,AVG_MICRO,0.890086,NaN,NaN
